# 🧪 Ground State Estimation using VQE for the ANNNI Model

This notebook is part of my SoQ project where I'm replicating part of the paper *'Quantum Convolutional Neural Network in ANNNI Model'*. 
Here, I’ve implemented the **Variational Quantum Eigensolver (VQE)** to estimate the ground state energy of the ANNNI Hamiltonian.

In [ ]:
# 📦 Import required libraries
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.opflow import PauliSumOp
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit.opflow.primitive_ops import PauliOp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ⚙️ Define the ANNNI Hamiltonian (Z.Z, Z..Z, and X terms)
def generate_annni_hamiltonian(n, kappa=0.5, h=1.0):
    terms = []
    for i in range(n - 1):
        z_term = ["I"] * n
        z_term[i] = "Z"
        z_term[i + 1] = "Z"
        terms.append(PauliOp.from_label("".join(z_term)))

    for i in range(n - 2):
        zz_term = ["I"] * n
        zz_term[i] = "Z"
        zz_term[i + 2] = "Z"
        terms.append(kappa * PauliOp.from_label("".join(zz_term)))

    for i in range(n):
        x_term = ["I"] * n
        x_term[i] = "X"
        terms.append(h * PauliOp.from_label("".join(x_term)))

    return sum(terms).reduce()

# ▶️ Generate Hamiltonian for 6 qubits
n_qubits = 6
hamiltonian = generate_annni_hamiltonian(n_qubits)
print("Hamiltonian constructed successfully.")

In [ ]:
# 🔧 Build a hardware-efficient ansatz
def hardware_ansatz(n, depth):
    circuit = QuantumCircuit(n)
    params = ParameterVector('θ', length=n * depth * 2)

    count = 0
    for d in range(depth):
        for i in range(n):
            circuit.ry(params[count], i)
            count += 1
        for i in range(n - 1):
            circuit.cx(i, i + 1)
        for i in range(n):
            circuit.ry(params[count], i)
            count += 1
    return circuit, params

depth = 3
ansatz, parameters = hardware_ansatz(n_qubits, depth)
ansatz.draw('mpl')

In [ ]:
# 🧮 Run VQE to estimate ground state energy
backend = Aer.get_backend('aer_simulator_statevector')
qi = QuantumInstance(backend=backend, shots=1024, seed_simulator=42, seed_transpiler=42)

vqe = VQE(ansatz=ansatz, optimizer=COBYLA(maxiter=200), quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)

print("Estimated Ground State Energy:", result.eigenvalue.real)

### ✅ Final Output
- Using VQE, we approximated the ground state energy of the ANNNI Hamiltonian.
- This forms the first step of the paper I'm replicating for SoQ.

**Note**: Accuracy can be improved further with deeper ansatz or better optimizers like SPSA or Adam.
